In [2]:
import os
os.chdir('/n/data1/hms/dbmi/oconnor/lab/shz311/pangenome/graph_var')

from graph_var.utils import load_graph_from_pkl, merge_dicts, log_action
from graph_var.graph import PangenomeGraph
from graph_var.evaluating_functions import *
import pandas as pd

version = 'v1'
ref_name = 'CHM13'

graph_obj_dir = f"/n/data1/hms/dbmi/oconnor/lab/shz311/pangenome/Graph_objs_{version}{'_chm13' if ref_name == 'CHM13' else ''}"
raw_vcf_dir = f"/n/data1/hms/dbmi/oconnor/lab/shz311/pangenome/VCFs_chr"
graph_vcf_dir = f"/n/data1/hms/dbmi/oconnor/lab/shz311/pangenome/VCFs_{version}{'_chm13' if ref_name == 'CHM13' else ''}"

ref_tree_dir = f"/n/data1/hms/dbmi/oconnor/lab/shz311/pangenome/Data/reference_tree_gfa_{version}{'_chm13' if ref_name == 'CHM13' else ''}"
gfa_dir = f"/n/data1/hms/dbmi/oconnor/lab/shz311/pangenome/Data/chromosome_gfa_{version}{'_chm13' if ref_name == 'CHM13' else ''}"
snarl_dir = f"/n/data1/hms/dbmi/oconnor/lab/shz311/pangenome/Data/chr_snarls_{version}{'_chm13' if ref_name == 'CHM13' else ''}"
bubble_summary_dir = f"/n/data1/hms/dbmi/oconnor/lab/shz311/pangenome/Bubble_summary_{version}{'_chm13' if ref_name == 'CHM13' else ''}"

var_summary_dir = f"/n/data1/hms/dbmi/oconnor/lab/shz311/pangenome/Stats_chr_{version}{'_chm13' if ref_name == 'CHM13' else ''}"
data_vis_dir = f"/n/data1/hms/dbmi/oconnor/lab/shz311/pangenome/Data_visualization_{version}{'_chm13' if ref_name == 'CHM13' else ''}"

region_dir = f"/n/data1/hms/dbmi/oconnor/lab/shz311/pangenome/Region_files"

mode = 'AT'
exclude_terminus = True
#lsdkhgsdlgh

In [3]:
chr_set = 'autosome'

if chr_set == 'autosome':
    chr_list = list(range(1, 23))
elif chr_set == 'X':
    chr_list = ['X']
elif chr_set == 'Y':
    chr_list = ['Y']
else:
    raise ValueError("chr_set must be one of ['autosome', 'X', 'Y']")

num_chr = len(chr_list)

Main Table

Supplementary Table

S1: Comprehensive variant summary table

In [4]:
import pandas as pd

graph_vcf_summary_df_list = []

for i in chr_list:
    graph_vcf_summary_df = pd.read_csv(f"{var_summary_dir}/comprehensive_variant_summary_for_chr{i}.csv")
    graph_vcf_summary_df_list.append(graph_vcf_summary_df)

def sum_up_dfs(df_list, num_id):
    combined = df_list[0].copy()
    for df in df_list[1:]:
        combined.iloc[:, num_id:] += df.iloc[:, num_id:]
    
    return combined

graph_vcf_all_df = sum_up_dfs(graph_vcf_summary_df_list, 2)
graph_vcf_all_df['CHROM'] = 'all'

graph_vcf_summary_df_list.append(graph_vcf_all_df)
graph_vcf_summary_concated_df = pd.concat(graph_vcf_summary_df_list)

In [5]:
graph_vcf_summary_concated_df

,CHROM,Variant Type,All_Variants,Linear_Variants,Off_Linear_Variants,Small_Variants,Large_Variants,Common_Variants,Uncommon_Variants,Linear_Small_Variants,...,Large_Common_Variants,Large_Uncommon_Variants,Linear_Small_Common_Variants,Linear_Large_Common_Variants,Linear_Small_Uncommon_Variants,Linear_Large_Uncommon_Variants,Off_Linear_Small_Common_Variants,Off_Linear_Large_Common_Variants,Off_Linear_Small_Uncommon_Variants,Off_Linear_Large_Uncommon_Variants
0,chr1,SNP,1921075,1709639,211436,1921075,0,727604,1193471,1709639,...,0,0,693343,0,1016296,0,34261,0,177175,0
1,chr1,MNP,58753,33961,24792,58697,56,13867,44886,33938,...,11,45,10950,9,22988,14,2906,2,21853,31
2,chr1,Insertion,277421,213998,63423,269966,7455,127021,150400,208409,...,2589,4866,104282,2256,104127,3333,20150,333,41407,1533
3,chr1,Deletion,306548,252371,54177,297708,8840,128849,177699,246652,...,2775,6065,112737,2185,133915,3534,13337,590,37719,2531
4,chr1,Replacement,30249,19590,10659,26112,4137,8576,21673,17438,...,845,3292,6366,597,11072,1555,1365,248,7309,1737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,all,Deletion,3875821,3145006,730815,3752194,123627,1602395,2273426,3066449,...,40208,83419,1393790,32346,1672659,46211,168397,7862,517348,37208
4,all,Replacement,396174,257689,138485,339269,56905,110228,285946,228023,...,11749,45156,82396,8942,145627,20724,16083,2807,95163,24432
5,all,Inversion,845,609,236,4,841,368,477,4,...,367,474,1,283,3,322,0,84,0,152
6,all,Duplication,65,60,5,0,65,14,51,0,...,14,51,0,14,0,46,0,0,0,5


In [7]:
file_path = f"{data_vis_dir}/supplementary_tables_{chr_set}.xlsx"

sheet_name = "S1"

if os.path.exists(file_path):
    with pd.ExcelFile(file_path) as xls:
        sheets = {sheet: pd.read_excel(xls, sheet) for sheet in xls.sheet_names}
else:
    sheets = {}

sheets[sheet_name] = graph_vcf_summary_concated_df

with pd.ExcelWriter(file_path, engine="openpyxl") as writer:
    for sheet, df in sheets.items():
        df.to_excel(writer, sheet_name=sheet, index=False)

S2: Tandom repeat for each variant type

In [8]:
import pandas as pd

graph_vcf_summary_df_list = []

for i in chr_list:
    graph_vcf_summary_df = pd.read_csv(f"{var_summary_dir}/comprehensive_repeated_variant_summary_for_chr{i}.csv")
    graph_vcf_summary_df_list.append(graph_vcf_summary_df)

def sum_up_dfs(df_list, num_id):
    combined = df_list[0].copy()
    for df in df_list[1:]:
        combined.iloc[:, num_id:] += df.iloc[:, num_id:]
    
    return combined

graph_vcf_all_df = sum_up_dfs(graph_vcf_summary_df_list, 2)
graph_vcf_all_df['CHROM'] = 'all'

graph_vcf_summary_df_list.append(graph_vcf_all_df)
graph_vcf_summary_concated_df = pd.concat(graph_vcf_summary_df_list)

In [9]:
graph_vcf_summary_concated_df

,CHROM,Variant Type,All_Variants,Linear_Variants,Off_Linear_Variants,Small_Variants,Large_Variants,Common_Variants,Uncommon_Variants,Linear_Small_Variants,...,Large_Common_Variants,Large_Uncommon_Variants,Linear_Small_Common_Variants,Linear_Large_Common_Variants,Linear_Small_Uncommon_Variants,Linear_Large_Uncommon_Variants,Off_Linear_Small_Common_Variants,Off_Linear_Large_Common_Variants,Off_Linear_Small_Uncommon_Variants,Off_Linear_Large_Uncommon_Variants
0,chr1,SNP,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,chr1,MNP,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,chr1,Insertion,239061,185206,53855,237864,1197,110558,128503,184232,...,506,691,91406,443,92826,531,18646,63,34986,160
3,chr1,Deletion,234726,196017,38709,233466,1260,105495,129231,195209,...,550,710,94004,436,101205,372,10941,114,27316,338
4,chr1,Replacement,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,all,Deletion,2913856,2405180,508676,2896538,17318,1294468,1619388,2393900,...,7434,9884,1150063,5953,1243837,5327,136971,1481,365667,4557
4,all,Replacement,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,all,Inversion,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,all,Duplication,65,60,5,0,65,14,51,0,...,14,51,0,14,0,46,0,0,0,5


In [10]:
file_path = f"{data_vis_dir}/supplementary_tables_{chr_set}.xlsx"

sheet_name = "S2"

if os.path.exists(file_path):
    with pd.ExcelFile(file_path) as xls:
        sheets = {sheet: pd.read_excel(xls, sheet) for sheet in xls.sheet_names}
else:
    sheets = {}

sheets[sheet_name] = graph_vcf_summary_concated_df

with pd.ExcelWriter(file_path, engine="openpyxl") as writer:
    for sheet, df in sheets.items():
        df.to_excel(writer, sheet_name=sheet, index=False)

S3: Variant type summary in regions (easy, segdup, hard)

In [99]:
var_region_concated_df = pd.read_csv(f"{data_vis_dir}/variant_summary_by_region_{chr_set}.csv")

In [100]:
file_path = f"{data_vis_dir}/supplementary_tables_{chr_set}.xlsx"

sheet_name = "S3"

if os.path.exists(file_path):
    with pd.ExcelFile(file_path) as xls:
        sheets = {sheet: pd.read_excel(xls, sheet) for sheet in xls.sheet_names}
else:
    sheets = {}

sheets[sheet_name] = var_region_concated_df

with pd.ExcelWriter(file_path, engine="openpyxl") as writer:
    for sheet, df in sheets.items():
        df.to_excel(writer, sheet_name=sheet, index=False)

S4: SNPs summary in regions compared between ourvcf and vcfwave

In [101]:
region_concated_df = pd.read_csv(f"{data_vis_dir}/snp_region_summary_ourvcf_vs_vcfwave_{chr_set}.csv")

In [102]:
file_path = f"{data_vis_dir}/supplementary_tables_{chr_set}.xlsx"

sheet_name = "S4"

if os.path.exists(file_path):
    with pd.ExcelFile(file_path) as xls:
        sheets = {sheet: pd.read_excel(xls, sheet) for sheet in xls.sheet_names}
else:
    sheets = {}

sheets[sheet_name] = region_concated_df

with pd.ExcelWriter(file_path, engine="openpyxl") as writer:
    for sheet, df in sheets.items():
        df.to_excel(writer, sheet_name=sheet, index=False)

S5: SNPs summary groupbed by minor allele count range compared between ourvcf and vcfwave

In [103]:
range_concated_df = pd.read_csv(f"{data_vis_dir}/snp_ac_range_summary_ourvcf_vs_vcfwave_{chr_set}.csv")

In [104]:
file_path = f"{data_vis_dir}/supplementary_tables_{chr_set}.xlsx"

sheet_name = "S5"

if os.path.exists(file_path):
    with pd.ExcelFile(file_path) as xls:
        sheets = {sheet: pd.read_excel(xls, sheet) for sheet in xls.sheet_names}
else:
    sheets = {}

sheets[sheet_name] = range_concated_df

with pd.ExcelWriter(file_path, engine="openpyxl") as writer:
    for sheet, df in sheets.items():
        df.to_excel(writer, sheet_name=sheet, index=False)

S6: Histogram of number of variant edges and number of ALT alleles in superbubbles

In [105]:
import pandas as pd

df_list = []
for i in chr_list:
    ac_df_path = f"{bubble_summary_dir}/bubble_allele_summary_chr{i}.tsv"
    ac_df = pd.read_csv(ac_df_path, sep='\t')
    df_list.append(ac_df)

concated_df = pd.concat(df_list)

variant_df = concated_df[concated_df.apply(lambda x: int(x['num_variants']) >= 2 and int(x['Level']) == 0, axis=1)]

def filter_level_multiallelic(x, colname):
    num_allele = x[colname]
    if num_allele == '.':
        return False
    level = x['Level']
    return int(num_allele) >= 2 and int(level) == 0

raw_vcf_allele_df = concated_df[concated_df.apply(lambda x: filter_level_multiallelic(x, 'raw_vcf_allele_count'), axis=1)]
vcfwave_allele_df = concated_df[concated_df.apply(lambda x: filter_level_multiallelic(x, 'vcfwave_allele_count'), axis=1)]

variant_data = variant_df['num_variants'].to_numpy().astype(int)
raw_vcf_allele_data = raw_vcf_allele_df['raw_vcf_allele_count'].to_numpy().astype(int)
vcfwave_allele_data = vcfwave_allele_df['vcfwave_allele_count'].to_numpy().astype(int)

In [106]:
# Define bins (Ensure both datasets use the same binning)
bins = [2, 3, 5, 7, 10, 13, 20] + [float('inf')]
bin_labels = ["2", "3-4", "5-6", "7-9", "10-12", "13-19", "20+"]

# Compute histograms
counts_edges, _ = np.histogram(variant_data, bins=bins)
counts_alt, _ = np.histogram(raw_vcf_allele_data, bins=bins)

In [107]:
grouped_bar_df = pd.DataFrame({
    "Ranges": bin_labels,
    "Variant edges": counts_edges,
    "ALT alleles": counts_alt
})
grouped_bar_df

,Ranges,Variant edges,ALT alleles
0,2,32216,32696
1,3-4,16471,16528
2,5-6,6233,6090
3,7-9,3999,4004
4,10-12,1781,1716
5,13-19,1497,1466
6,20+,1324,1005


In [108]:
file_path = f"{data_vis_dir}/supplementary_tables_{chr_set}.xlsx"

sheet_name = "S6"

if os.path.exists(file_path):
    with pd.ExcelFile(file_path) as xls:
        sheets = {sheet: pd.read_excel(xls, sheet) for sheet in xls.sheet_names}
else:
    sheets = {}

sheets[sheet_name] = grouped_bar_df

with pd.ExcelWriter(file_path, engine="openpyxl") as writer:
    for sheet, df in sheets.items():
        df.to_excel(writer, sheet_name=sheet, index=False)

S7: Scatterplot of frequency of pair (number of variant edge, number of ALT alleles)

In [109]:
import pandas as pd

df_list = []
for i in chr_list:
    ac_df_path = f"{bubble_summary_dir}/bubble_allele_summary_chr{i}.tsv"
    ac_df = pd.read_csv(ac_df_path, sep='\t')
    df_list.append(ac_df)

concated_df = pd.concat(df_list)

level0_df = concated_df[concated_df.apply(lambda x: int(x['Level']) == 0 and x['raw_vcf_allele_count'] != '.', axis=1)]

from collections import Counter

# Convert to numpy arrays
x = np.asarray(level0_df['num_variants']).astype(int)
y = np.asarray(level0_df['raw_vcf_allele_count']).astype(int)

# Filter by range
min_range = 0
max_range = 100
sample_index = (x >= min_range) & (x <= 3 * max_range) & (y >= min_range) & (y <= max_range)
x = x[sample_index]
y = y[sample_index]

# Count occurrences of each unique (x, y) pair
coords = list(zip(x, y))
count_dict = Counter(coords)

# Extract unique coords and their counts
num_vars = [a for a, _ in count_dict.keys()]
num_alleles = [b for _, b in count_dict.keys()]
counts = list(count_dict.values())

scatter_df = pd.DataFrame({
    "Variant edges": num_vars,
    "ALT alleles": num_alleles,
    "Counts": counts
})

In [110]:
file_path = f"{data_vis_dir}/supplementary_tables_{chr_set}.xlsx"

sheet_name = "S7"

if os.path.exists(file_path):
    with pd.ExcelFile(file_path) as xls:
        sheets = {sheet: pd.read_excel(xls, sheet) for sheet in xls.sheet_names}
else:
    sheets = {}

sheets[sheet_name] = scatter_df

with pd.ExcelWriter(file_path, engine="openpyxl") as writer:
    for sheet, df in sheets.items():
        df.to_excel(writer, sheet_name=sheet, index=False)

S8: Summary of triallelic superbubbles

In [111]:
from collections import defaultdict

# Initialize a defaultdict to count bubble types across files
bubble_type_counts = defaultdict(int)

# Assuming all files are in a directory, you can use glob to get the file paths
file_dir = bubble_summary_dir

# Iterate over each file
for i in range(1,23):
    # Read the file into a DataFrame
    file_path = f"{file_dir}/bubble_triallelic_chr{i}.tsv"
    df = pd.read_csv(file_path, sep='\t')  # Assuming tab-separated values, adjust if necessary
    
    # Count the occurrences of each Bubble_type
    bubble_type_count = df['Bubble_type'].value_counts()
    
    # Update the overall bubble type count
    for bubble_type, count in bubble_type_count.items():
        if bubble_type not in {'overlapping', 'interlocking', 'nested', 'properly_triallelic'}:
            continue
        bubble_type_counts[bubble_type] += count
        
# Convert the defaultdict to a pandas DataFrame for plotting
bubble_type_df = pd.DataFrame(list(bubble_type_counts.items()), columns=['Bubble_type', 'Count'])

In [112]:
file_path = f"{data_vis_dir}/supplementary_tables_{chr_set}.xlsx"

sheet_name = "S8"

if os.path.exists(file_path):
    with pd.ExcelFile(file_path) as xls:
        sheets = {sheet: pd.read_excel(xls, sheet) for sheet in xls.sheet_names}
else:
    sheets = {}

sheets[sheet_name] = bubble_type_df

with pd.ExcelWriter(file_path, engine="openpyxl") as writer:
    for sheet, df in sheets.items():
        df.to_excel(writer, sheet_name=sheet, index=False)

S9: Summary table of triallelic variants

In [113]:
var_dict = {}

for i in tqdm(chr_list):
    tri_var_path = f"{bubble_summary_dir}/triallelic_variants_chr{i}.tsv"
    tri_var_df = pd.read_csv(tri_var_path, sep="\t")
    for j in range(len(tri_var_df)):
        info_dict = get_info_dict(tri_var_df.loc[j, "INFO"])
        VT = info_dict['VT']
        # var_dict[VT] = var_dict.get(VT, 0) + 1
        linear = int(info_dict['DR'].split(',')[1]) == 0

        ref = tri_var_df.loc[j, "REF"] if tri_var_df.loc[j, "REF"] != '.' else info_dict['NR']
        alt = tri_var_df.loc[j, "ALT"]

        length = len(ref) + len(alt)

        if length < 50:
            var_dict['small_'+VT] = var_dict.get('small_'+VT, 0) + 1
        else:
            var_dict['large_'+VT] = var_dict.get("large_"+VT, 0) + 1
        
        if linear:
            var_dict['linear_'+VT] = var_dict.get('linear_'+VT, 0) + 1
        else:
            var_dict['off-linear_'+VT] = var_dict.get('off-linear_'+VT, 0) + 1

100%|██████████| 1/1 [00:02<00:00,  2.42s/it]


In [114]:
categories = ['SNPs', 'MNPs', 'Insertions', 'Deletions', 'Replacements', 'Duplications', 'Inversions']
small_list = [var_dict.get('small_'+vt[:3].upper(), 0) for vt in categories]
large_list = [var_dict.get('large_'+vt[:3].upper(), 0) for vt in categories]
linear_list = [var_dict.get('linear_'+vt[:3].upper(), 0) for vt in categories]
off_linear_list = [var_dict.get('off-linear_'+vt[:3].upper(), 0) for vt in categories]
total_list = [s+l for s, l in zip(small_list, large_list)]

tri_var_sum_df = pd.DataFrame({
    "Variant Type": categories,
    "Small": small_list,
    "Large": large_list,
    "Linear": linear_list,
    "Off-linear": off_linear_list,
    "Total": total_list
})

In [115]:
file_path = f"{data_vis_dir}/supplementary_tables_{chr_set}.xlsx"

sheet_name = "S9"

if os.path.exists(file_path):
    with pd.ExcelFile(file_path) as xls:
        sheets = {sheet: pd.read_excel(xls, sheet) for sheet in xls.sheet_names}
else:
    sheets = {}

sheets[sheet_name] = tri_var_sum_df

with pd.ExcelWriter(file_path, engine="openpyxl") as writer:
    for sheet, df in sheets.items():
        df.to_excel(writer, sheet_name=sheet, index=False)

In [116]:
tri_var_sum_df

,Variant Type,Small,Large,Linear,Off-linear,Total
0,SNPs,22785,0,21213,1572,22785
1,MNPs,1961,0,1895,66,1961
2,Insertions,20202,229,17923,2508,20431
3,Deletions,18139,180,17265,1054,18319
4,Replacements,917,19,904,32,936
5,Duplications,0,0,0,0,0
6,Inversions,0,0,0,0,0


S10: Types of muti-allelic superbubbles and variants summary

In [117]:
import numpy as np
from collections import defaultdict

def check_ac_range(ac):
    assert ac >= 1
    bins = [1, 2, 3, 5, 7, 10, 13, 20, np.inf]
    bin_labels = ["1", "2", "3-4", "5-6", "7-9", "10-12", "13-19", "20+"]

    idx = None
    for i in range(len(bins)):
        if bins[i] <= ac and ac < bins[i+1]:
            idx = i
            break
    
    return bin_labels[idx] if idx is not None else None

def recursion_dict():
    return defaultdict(recursion_dict)

In [118]:
import gc

superbubble_dict = recursion_dict()
superbubble_with_tr_dict = recursion_dict()
variant_dict = recursion_dict()

for i in tqdm(chr_list):
    graph_vcf_path = f"{graph_vcf_dir}/graph_chr{i}{'_no_terminus' if exclude_terminus else ''}.vcf"
    bubble_ac_path = f"{bubble_summary_dir}/bubble_allele_summary_chr{i}.tsv"
    bubble_AT_path = f"{bubble_summary_dir}/bubble_variant_counts_chr{i}_AT.tsv"
    bubble_type_path = f"{bubble_summary_dir}/superbubble_type_chr{i}.tsv"

    var_dict = {}

    for row in read_vcf_line_by_line(graph_vcf_path):
        edge = extract_bubble_ids(row['ID'], symbol=True)
        info_dict = get_info_dict(row['INFO'])
        info_dict['POS'] = int(row['POS'])

        var_dict[edge] = info_dict

    bubble_ac_df = pd.read_csv(bubble_ac_path, sep='\t')
    bubble_ac_dict = {}

    for row in bubble_ac_df.itertuples(index=False):
        if int(row.Level) != 0: 
            continue
        bubble = tuple(sorted(eval(row.Bubble)))

        ac = int(row.raw_vcf_allele_count) if row.raw_vcf_allele_count != '.' else 0

        if ac >= 1:
            bubble_ac_dict[bubble] = ac

    del bubble_ac_df
    gc.collect()

    bubble_AT_df = pd.read_csv(bubble_AT_path, sep='\t')
    bubble_within_dict = {}

    for row in bubble_AT_df.itertuples(index=False):
        bubble = tuple(sorted(eval(row.Bubble)))

        if int(row.Level) != 0:
            continue

        within = eval(row.Within)

        bubble_within_dict[bubble] = within

    del bubble_AT_df
    gc.collect()
    
    bubble_type_df = pd.read_csv(bubble_type_path, sep='\t')
    bubble_type_dict = {}

    for row in bubble_type_df.itertuples(index=False):
        bubble = tuple(sorted(eval(row.Bubble)))

        bubble_type_dict[bubble] = row.Type

    del bubble_type_df
    gc.collect()

    for bubble in bubble_ac_dict.keys():
        ac_range = check_ac_range(bubble_ac_dict[bubble])
        bubble_type = bubble_type_dict[bubble]
        within_vars = bubble_within_dict[bubble]

        if bubble_type in superbubble_dict and ac_range in superbubble_dict[bubble_type]:
            superbubble_dict[bubble_type][ac_range] += 1
        else:
            superbubble_dict[bubble_type][ac_range] = 1
        
        TR_flags = []
        for var in within_vars:
            linearity = "GRCh38" if var_dict[var]['NR'] == '.' else "Non-GRCh38"
            TR = var_dict[var]['TR_MOTIF'] != '.'
            TR_flags.append(TR)

            TR_label = "TR" if var_dict[var]['TR_MOTIF'] != '.' else "Non-TR"

            if (bubble_type in variant_dict 
                and ac_range in variant_dict[bubble_type] 
                and linearity in variant_dict[bubble_type][ac_range]):
                variant_dict[bubble_type][ac_range][linearity] += 1
            else:
                variant_dict[bubble_type][ac_range][linearity] = 1
            
            if (bubble_type in variant_dict 
                and ac_range in variant_dict[bubble_type] 
                and TR_label in variant_dict[bubble_type][ac_range]):
                variant_dict[bubble_type][ac_range][TR_label] += 1
            else:
                variant_dict[bubble_type][ac_range][TR_label] = 1

        if any(TR_flags):
            if (bubble_type in superbubble_with_tr_dict 
                and ac_range in superbubble_with_tr_dict[bubble_type]):
                superbubble_with_tr_dict[bubble_type][ac_range] += 1
            else:
                superbubble_with_tr_dict[bubble_type][ac_range] = 1

superbubble_summary_df = pd.DataFrame(columns=["Superbubble type", 
                                               "Superbubble ALT allele group", 
                                               "Superbubble count",
                                               "Superbubble with TR count",
                                               "Variants count (TR)",
                                               "Variants count (Non-TR)",
                                               "Variants count (GRCh38)",
                                               "Variants count (Non-GRCh38)"])

for bubble_type in superbubble_dict.keys():
    for ac_range in superbubble_dict[bubble_type].keys():
        superbubble_count = superbubble_dict[bubble_type].get(ac_range, 0)
        superbubble_tr_count = superbubble_with_tr_dict[bubble_type].get(ac_range, 0)
        var_count_TR = variant_dict[bubble_type][ac_range].get('TR', 0)
        var_count_nonTR = variant_dict[bubble_type][ac_range].get('Non-TR', 0)
        var_count_linear = variant_dict[bubble_type][ac_range].get('GRCh38', 0)
        var_count_offlinear = variant_dict[bubble_type][ac_range].get('Non-GRCh38', 0)

        superbubble_summary_df.loc[len(superbubble_summary_df)] = [bubble_type, 
                                                                   ac_range, 
                                                                   superbubble_count,
                                                                   superbubble_tr_count,
                                                                   var_count_TR,
                                                                   var_count_nonTR,
                                                                   var_count_linear,
                                                                   var_count_offlinear]

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:55<00:00, 55.74s/it]


In [119]:
superbubble_summary_df.sort_values(by=["Superbubble type", "Superbubble ALT allele group"], inplace=True)
superbubble_summary_df

,Superbubble type,Superbubble ALT allele group,Superbubble count,Superbubble with TR count,Variants count (TR),Variants count (Non-TR),Variants count (GRCh38),Variants count (Non-GRCh38)
9,deletion,1,50572,35598,35598,14974,50572,0
15,deletion,10-12,1104,1092,10619,1832,11006,1445
14,deletion,13-19,845,835,10474,3822,11835,2461
8,deletion,2,13140,11830,22073,4357,26430,0
13,deletion,20+,303,298,8073,15186,19223,4036
10,deletion,3-4,10376,10072,32257,3344,34590,1011
12,deletion,5-6,4227,4148,21144,2114,21799,1459
11,deletion,7-9,2748,2709,19445,2798,20353,1890
17,insertion,1,47962,41088,41108,6902,47985,25
22,insertion,10-12,162,158,1403,1624,1065,1962


In [120]:
file_path = f"{data_vis_dir}/supplementary_tables_{chr_set}.xlsx"

sheet_name = "S10"

if os.path.exists(file_path):
    with pd.ExcelFile(file_path) as xls:
        sheets = {sheet: pd.read_excel(xls, sheet) for sheet in xls.sheet_names}
else:
    sheets = {}

sheets[sheet_name] = superbubble_summary_df

with pd.ExcelWriter(file_path, engine="openpyxl") as writer:
    for sheet, df in sheets.items():
        df.to_excel(writer, sheet_name=sheet, index=False)